In [23]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import player_stats

In [94]:
def full_player_stats(competition, year):
    
    import pandas as pd
    from internal_packages import check_comp
    import requests
    from bs4 import BeautifulSoup
    
    player_data_types = ['stats',  'shooting', 'passing', 'passing_types', 'gca', 'defense', 'possession', 'playingtime', 'misc']
    
    df = pd.DataFrame()
        
    try:
        comp = check_comp.check_comp(competition) ## Checking competitions with pre-created formula
        season = f"{str(year)}-{str(year+1)}" ## Creating proper season string
        
        for d_type in player_data_types:
            url = f"https://fbref.com/en/comps/{comp[0]}/{season}/{d_type}/{season}-{comp[1]}-Stats" ## Creating the url
            resp = requests.get(url) 
            print(resp)
            resp.encoding = 'utf-8'
            soup = BeautifulSoup(resp.text.replace('<!--', '').replace('--!>', ''), 'html.parser')
            
            ## Creting two lists which will be filled with data

            headers, rows = [], []

            ## Getting values from the very first row of the table, from which with iterration we will get all the values from 'data-stat' attributes, which will be used as headers

            table_data = soup.find_all('tbody')[2]
            table_rows = table_data.find_all('tr')

            ## Getting the headers and putting them into the headers list

            for value in table_rows[0]:
                headers.append(value.get('data-stat'))

            ## Getting every row value and putting them into the rows list

            for row in table_rows:
                row_values = []
                row_data = row.find_all('td')
                for data_value in row_data:
                    if data_value.get('data-append-csv'):
                        row_values.append(data_value.get('data-append-csv')) ## 'data-append-csv' is an attribute inside of each row representing the unique code for every player, which will be used as 'player_id' later
                    row_values.append(data_value.get_text())
                rows.append(row_values)

            ## Creating the dataframe

            data_df = pd.DataFrame(rows, columns = headers).fillna(0) ## Creating the dataframe from the data
            data_df = data_df.loc[data_df['player'] != 0].reset_index(drop = True) ## FBRef has an unused row every approx 25 rows which is filled with NaN. This code is to remove it, while immediately reseting the index to ease further iterrations
            
            if 'minutes' in headers:
                data_df['minutes'] = data_df['minutes'].str.replace(',', '')
            elif 'gk_minutes' in headers:
                data_df['gk_minutes'] = data_df['gk_minutes'].str.replace(',', '')

            ## For the actual season, age column always shows the age in format yy-ddd, with the players age exactly shown in years and days
            ## Following algorithm is used to remove dash and ddd, so data can be transformed into numerical value

            for i, row in data_df.iterrows():
                try:
                    if '-' in row['age']: ## If '-' exist in the column
                        age_num = row['age'].split('-')[0] ## Splitting the string where the '-' is and keeping only the part before it (years of age), storing it into an age_num variable
                        data_df.at[i, 'age'] = age_num ## Changing values in every row where column is age with the age_num variable
                except Exception: pass ## Ignoring if data transformation is not necessary
            
            data_df.rename(columns={'ranker' : 'player_id'}, inplace=True) ## Renaming the first column from ranker to player_id for easier understading
            data_df = data_df.apply(pd.to_numeric, errors = 'ignore') ## Transforming all possible values from object into numerical values, ignoring the ones that cannot be changed instead of deleting or changing them
            
            data_df = data_df.set_index('player_id')
            
            df = pd.merge(df, data_df, left_index=True, right_index=True, how='inner')
        
        df = df.loc[:,~df.columns.duplicated()]
        
        df = df.reset_index()
        
        return df
            
    except: return "Invalid data input" ## Returning string what shows invalid input

In [95]:
df = full_player_stats("eng", 2024)

<Response [200]>
Empty DataFrame
Columns: [player, nationality, position, team, age, birth_year, games, games_starts, minutes, minutes_90s, goals, assists, goals_assists, goals_pens, pens_made, pens_att, cards_yellow, cards_red, xg, npxg, xg_assist, npxg_xg_assist, progressive_carries, progressive_passes, progressive_passes_received, goals_per90, assists_per90, goals_assists_per90, goals_pens_per90, goals_assists_pens_per90, xg_per90, xg_assist_per90, xg_xg_assist_per90, npxg_per90, npxg_xg_assist_per90, matches]
Index: []

[0 rows x 36 columns]
<Response [200]>
Empty DataFrame
Columns: [player_x, nationality_x, position_x, team_x, age_x, birth_year_x, games, games_starts, minutes, minutes_90s_x, goals_x, assists, goals_assists, goals_pens, pens_made_x, pens_att_x, cards_yellow, cards_red, xg_x, npxg_x, xg_assist, npxg_xg_assist, progressive_carries, progressive_passes, progressive_passes_received, goals_per90, assists_per90, goals_assists_per90, goals_pens_per90, goals_assists_pens_pe

In [89]:
df

'Invalid data input'

In [61]:
stats = player_stats.player_stats('eng', 2024, 'stats')
playingtime = player_stats.player_stats('eng', 2024, 'playingtime')
misc = player_stats.player_stats('eng', 2024, 'misc')

In [62]:
stats = stats.set_index('player_id')
playingtime = playingtime.set_index('player_id')
misc = misc.set_index('player_id')

In [63]:
import pandas as pd

In [84]:
stats

,player,nationality,position,team,age,birth_year,games,games_starts,minutes,minutes_90s,...,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg_per90,xg_assist_per90,xg_xg_assist_per90,npxg_per90,npxg_xg_assist_per90,matches
player_id,,,,,,,,,,,,,,,,,,,,,
774cf58b,Max Aarons,eng ENG,DF,Bournemouth,25,2000,3,1,86,1.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
b7e62e1d,Joshua Acheampong,eng ENG,DF,Chelsea,18,2006,2,2,160,1.8,...,0.00,0.00,0.00,0.00,0.12,0.00,0.12,0.12,0.12,Matches
2b09d998,Tyler Adams,us USA,MF,Bournemouth,25,1999,13,8,776,8.6,...,0.35,0.35,0.00,0.35,0.12,0.08,0.21,0.12,0.21,Matches
c81d773d,Tosin Adarabioyo,eng ENG,DF,Chelsea,27,1997,11,8,733,8.1,...,0.00,0.12,0.12,0.12,0.07,0.00,0.07,0.07,0.07,Matches
4dcec659,Simon Adingra,ci CIV,"FW,MF",Brighton,23,2002,15,6,553,6.1,...,0.16,0.49,0.33,0.49,0.23,0.15,0.38,0.23,0.38,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
be927d03,Ashley Young,eng ENG,DF,Everton,39,1985,19,17,1472,16.4,...,0.18,0.24,0.06,0.24,0.01,0.09,0.10,0.01,0.10,Matches
88968486,Illia Zabarnyi,ua UKR,DF,Bournemouth,22,2002,23,23,2070,23.0,...,0.00,0.00,0.00,0.00,0.04,0.02,0.06,0.04,0.06,Matches
51cf8561,Oleksandr Zinchenko,ua UKR,"DF,MF",Arsenal,28,1996,8,2,243,2.7,...,0.00,0.00,0.00,0.00,0.02,0.01,0.03,0.02,0.03,Matches


In [77]:
playingtime

,player,nationality,position,team,age,birth_year,games,minutes,minutes_per_game,minutes_pct,...,on_goals_against,plus_minus,plus_minus_per90,plus_minus_wowy,on_xg_for,on_xg_against,xg_plus_minus,xg_plus_minus_per90,xg_plus_minus_wowy,matches
player_id,,,,,,,,,,,,,,,,,,,,,
774cf58b,Max Aarons,eng ENG,DF,Bournemouth,25.0,2000.0,3,86.0,29.0,4.2,...,0.0,2.0,2.09,1.50,2.3,0.3,2.0,2.12,1.52,Matches
c103921a,Zach Abbott,eng ENG,DF,Nott'ham Forest,18.0,2006.0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches
b7e62e1d,Joshua Acheampong,eng ENG,DF,Chelsea,18.0,2006.0,2,160.0,80.0,7.7,...,3.0,-1.0,-0.56,-1.32,3.4,2.1,1.3,0.73,0.13,Matches
2b09d998,Tyler Adams,us USA,MF,Bournemouth,25.0,1999.0,13,776.0,60.0,37.5,...,12.0,12.0,1.39,1.18,18.3,11.5,6.8,0.78,0.19,Matches
c81d773d,Tosin Adarabioyo,eng ENG,DF,Chelsea,27.0,1997.0,11,733.0,67.0,35.4,...,9.0,8.0,0.98,0.51,18.8,11.7,7.1,0.88,0.42,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88968486,Illia Zabarnyi,ua UKR,DF,Bournemouth,22.0,2002.0,23,2070.0,90.0,100.0,...,26.0,15.0,0.65,NaN,44.0,28.7,15.3,0.67,NaN,Matches
51cf8561,Oleksandr Zinchenko,ua UKR,"DF,MF",Arsenal,28.0,1996.0,8,243.0,30.0,11.7,...,0.0,3.0,1.11,0.13,2.7,1.8,0.9,0.32,-0.49,Matches
028e70b9,Joshua Zirkzee,nl NED,"FW,MF",Manchester Utd,23.0,2001.0,23,864.0,38.0,41.7,...,14.0,4.0,0.42,1.01,17.4,15.8,1.6,0.16,0.36,Matches


In [78]:
misc

,player,nationality,position,team,age,birth_year,minutes_90s,cards_yellow,cards_red,cards_yellow_red,...,interceptions,tackles_won,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,matches
player_id,,,,,,,,,,,,,,,,,,,,,
774cf58b,Max Aarons,eng ENG,DF,Bournemouth,25,2000,1.0,0,0,0,...,1,2,0,0,0,7,0,0,NaN,Matches
b7e62e1d,Joshua Acheampong,eng ENG,DF,Chelsea,18,2006,1.8,0,0,0,...,1,1,0,0,0,7,1,6,14.3,Matches
2b09d998,Tyler Adams,us USA,MF,Bournemouth,25,1999,8.6,4,0,0,...,23,29,0,1,0,39,13,7,65.0,Matches
c81d773d,Tosin Adarabioyo,eng ENG,DF,Chelsea,27,1997,8.1,2,0,0,...,3,9,0,0,0,24,19,12,61.3,Matches
4dcec659,Simon Adingra,ci CIV,"FW,MF",Brighton,23,2002,6.1,0,0,0,...,6,6,0,0,0,30,3,1,75.0,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
be927d03,Ashley Young,eng ENG,DF,Everton,39,1985,16.4,5,1,0,...,21,25,0,0,0,58,8,10,44.4,Matches
88968486,Illia Zabarnyi,ua UKR,DF,Bournemouth,22,2002,23.0,3,0,0,...,23,24,0,0,0,109,53,25,67.9,Matches
51cf8561,Oleksandr Zinchenko,ua UKR,"DF,MF",Arsenal,28,1996,2.7,1,0,0,...,3,0,0,0,0,13,6,6,50.0,Matches
